In [10]:
from flask import Flask, request
import openai
import os
import langchain
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, APIChain, ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import TextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, TextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.memory import ConversationSummaryMemory
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.chains.mapreduce import MapReduceChain
from langchain.document_loaders import Docx2txtLoader
from langchain.callbacks import get_openai_callback

In [4]:
REPHRASE_PROMPT_TEMPLATE = """With the background information \n {background} \n, now you are given the detailed job description \n {job} \n.
    Based on the background information and job description, generate a strong resume.
    Using the job description, select ONLY the top three related strong experience/projects as the background information, rephrase/summarize background to be much stronger and more match to the job. Keep at least three expriences and also project remained.
    DO NOT introduce non-existing information. Pay attention to deliberately exclude any unnecessary pieces of information in the generated resume. The generated resume should have length around 500 tokens, i.e. keep the resume concise on language and detailed in experience.
    
    generated resume:"""

LATEX_PROMPT_TEMPLAT_1 = """With the background information {background}, now you are given the latex template {template}.
    Based on the background information, fill corresonding information into the heading, education, and experience of the latex template.
    Using the background information, modifying as little as possible, keep the latex structure as unchanged, only modifying the text part, and generate a one-page latex pdf. 
    The experience section of the latex template refers to the working experience. The generated resume should ONLY include the heading, education, and the experience. Pay attention to deliberately exclude any unnecessary pieces of information in the generated resume: DO NOT include any projects and/or skills.
    Remember that in latex, we put the backslash '\' before the special characters, such as '#'. Use only utf-8 characters.
    
    generated resume:"""

LATEX_PROMPT_TEMPLAT_2 = """With the background information {background}, now you are given the latex template {template}.
    Based on the background information, fill corresonding information into the projects, programming skills, and ending of the latex template.
    Using the background information, modifying as little as possible, keep the latex structure as unchanged, only modifying the text part, and generate a one-page latex pdf. 
    The projects section of the latex template refers to the projects experience. The generated resume should ONLY include the projects, programming skills, and ending. Pay attention to deliberately exclude any unnecessary pieces of information in the generated resume: DO NOT include any heading, education and/or working experience.
    Remember that in latex, we put the backslash '\' before the special characters, such as '#' (e.g. C# should write as C\#).
    Use only utf-8 characters.
    Remeber that you are writing the second half part of the resume, so:
    You must include the \end document code to make sure the latex code is end.
    You must not include the \documentclass and \begin docment in the result. 

    generated resume:"""

In [22]:
with open('./resources/header.tex') as f:
    latex_header = f.read()

with open('./resume/part1.tex') as f:
    part1 = f.read()

with open('./resume/part2.tex') as f:
    part2 = f.read()

llm = OpenAI(model_name='text-davinci-003', max_tokens=1200)

background_path = './resources/Resume Materials.docx'

loader = Docx2txtLoader(background_path)
background_info = loader.load()

In [15]:
text_splitter = CharacterTextSplitter(chunk_size=4000, chunk_overlap=0)

background_info_doc = text_splitter.split_documents(background_info)

In [16]:
rephrase_template = PromptTemplate(
    input_variables=[
        "background",
        "job",
    ],
    template=REPHRASE_PROMPT_TEMPLATE,
)

rephrase_chain = LLMChain(llm=llm,
                          prompt=rephrase_template,
                          verbose=False)

results = []
for part in background_info_doc:
    with get_openai_callback() as cb:
        res = rephrase_chain.run(background=part, job=job_description)
        results.append(res)
        print(cb)

Tokens Used: 2953
	Prompt Tokens: 2058
	Completion Tokens: 895
Successful Requests: 1
Total Cost (USD): $0.059059999999999994
Tokens Used: 2488
	Prompt Tokens: 1749
	Completion Tokens: 739
Successful Requests: 1
Total Cost (USD): $0.049760000000000006


In [17]:
background_info_generated = '\n\n'.join(results)

In [18]:
results = []

reformat_template = PromptTemplate(
    input_variables=[
        "template",
        "background",
    ],
    template=LATEX_PROMPT_TEMPLAT_1,
)

reformat_chain = LLMChain(llm=llm, 
                          prompt=reformat_template,
                          verbose=False)

with get_openai_callback() as cb:
    res = reformat_chain.run(background=background_info_generated, template=part1)
    results.append(bytes(res, 'utf-8').decode('utf-8', 'ignore').strip())
    print(cb)

reformat_template = PromptTemplate(
    input_variables=[
        "template",
        "background",
    ],
    template=LATEX_PROMPT_TEMPLAT_2,
)

reformat_chain = LLMChain(llm=llm, 
                          prompt=reformat_template,
                          verbose=False)

with get_openai_callback() as cb:
    res = reformat_chain.run(background=background_info_generated, template=part2)
    results.append(bytes(res, 'utf-8').decode('utf-8', 'ignore').strip())
    print(cb)

Tokens Used: 3530
	Prompt Tokens: 2746
	Completion Tokens: 784
Successful Requests: 1
Total Cost (USD): $0.0706
Tokens Used: 3042
	Prompt Tokens: 2246
	Completion Tokens: 796
Successful Requests: 1
Total Cost (USD): $0.060840000000000005


In [19]:
generated_resume = latex_header + '\n' + results[0] + '\n' + results[1]

with open('./resume/resume.tex', 'w', encoding='utf-8') as f:
    f.write(generated_resume)

In [ ]:
os.system("pdflatex ./resume/resume.tex")

In [39]:
CHAT_PROMPT_TEMPLAT = """Based on the user request, identify whether the user is requesting to modify the resume.
    You are only allowed to answer in 'YES' or 'NO'. Do not include any other information or responses.

    User Request:{query}
    Response:"""

In [68]:
user_query = 'regenerate'

with open('./prompts/chat_template.txt') as f:
    CHAT_PROMPT_TEMPLAT = f.read()

chat_template = PromptTemplate(
    input_variables=["query"],
    template=CHAT_PROMPT_TEMPLAT,
)

chat_memory = ConversationBufferMemory(input_key='query', memory_key='query_history')

conversation = LLMChain(llm=OpenAI(temperature=0), memory=chat_memory, prompt=chat_template)

res = conversation.run(query=user_query).strip()

res

'YES'

In [69]:
with open('./prompts/revision_template.txt') as f:
    REVISE_PROMPT_TEMPLAT = f.read()

revise_template = PromptTemplate(
    input_variables=["old_template", "user_request"],
    template=REVISE_PROMPT_TEMPLAT,
)

with open('./prompts/rephrase_template.txt') as f:
    REPHRASE_PROMPT_TEMPLATE = f.read().replace('{', '#(#').replace('}', '#)#')
with open('./prompts/latex_template_1.txt') as f:
    LATEX_PROMPT_TEMPLAT_1 = f.read().replace('{', '#(#').replace('}', '#)#')
with open('./prompts/latex_template_2.txt') as f:
    LATEX_PROMPT_TEMPLAT_2 = f.read().replace('{', '#(#').replace('}', '#)#')

revision_chain = LLMChain(llm=OpenAI(temperature=0.7), 
                          prompt=revise_template,
                          verbose=False)

REPHRASE_PROMPT_TEMPLATE = revision_chain.run(old_template=REPHRASE_PROMPT_TEMPLATE, user_request=user_query)
LATEX_PROMPT_TEMPLAT_1 = revision_chain.run(old_template=LATEX_PROMPT_TEMPLAT_1, user_request=user_query)
LATEX_PROMPT_TEMPLAT_2 = revision_chain.run(old_template=LATEX_PROMPT_TEMPLAT_2, user_request=user_query)

with open('./prompts/rephrase_template.txt', 'w') as f:
    f.write(REPHRASE_PROMPT_TEMPLATE.replace('#(#', '{').replace('#)#', '}'))
with open('./prompts/latex_template_1.txt', 'w') as f:
    f.write(LATEX_PROMPT_TEMPLAT_1.replace('#(#', '{').replace('#)#', '}'))
with open('./prompts/latex_template_2.txt', 'w') as f:
    f.write(LATEX_PROMPT_TEMPLAT_2.replace('#(#', '{').replace('#)#', '}'))